<img src="https://i.ibb.co/NjSZDm5/kolkata-skyline-in-watercolor-background-pablo-romero.jpg" alt="kolkata"
	title="Kolkata" width = 1000/>

# Capstone Project - Kolkata - (The City of Joy) Restaurant Business Analysis 
### Applied Data Science Capstone by IBM/Coursera
### By Victor Banerjee


## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In the project I tried to find an optimal location for a restaurant. The stakeholders is very new in the restaurant business so they are totally depends on my analysis to find a **optimal location**  to start new restaurant chain business.  

As **Kolkata,IN** is one of the four major cities in India the most part of the **Kolkata,IN** is crowded with multiple restaurants. As it is a 300 year old cities lots of types cuisine restaurant are predominant in Kolkata. My goal is to **provide few optimal options to the stakeholders to run a profitable business restaurant**.
I will try to find the optimal place with with following criteria :

    1. Less crowded with popular restaurants.
    2. Customer Rating Of the restaurant
    3. Possible cuisine for the restaurant
    4. Highly popular place in Kolkata 

In the final step I will try to find few promising areas in Kolkata meet the stakeholders criteria. Using **E**xploratory **D**ata **A**nalysis with provide summary of Kolkata restaurant business paradigm.

# Data <a name="data"></a>

By understanding our current business problem I have outline few factors that the decisions are:
* Popularity of the Place in Kolkata and distance from it
* Numbers of and distance to the Popular restaurant are in the neighborhood
* What types and kinds of cuisines available in the Kolkata restaurants
* Customers sentiments on restaurant business

To resolve the business problem following datasets are needed to extract/ generate data to summarize our prospective:
- Kolkata's pin codes Coordinates are extract from Google Search Page by BeutifulSoup Library
- FourSquare Api is used to generate neighborhood datasets
- Zomato Api is used to generate restaurants details datasets
- Zomato Api is used for generating Customers reviews datasets

Python Library Used
~~~
from bs4 import BeautifulSoup #Web Scrapping
import numpy as np
from tabulate import tabulate
import json #library for Json file
from geopy.geocoders import Nominatim # Convert an address into latitude and longitude values
import folium # Map rendering library
~~~


## Methodology <a name="methodology"></a>

- I have used **BeautifulSoup** Kolkata's pincodes Coordinates scrap data from Google search page.
- There are no python codes available for extracting Zomato restaurants details, for I have created a **Object class for python to request Zomato Api**. 
- Data cleaning and EDA is done using Python Pandas and Numpy Library
- Folium Map used for displaying spatial location of the restaurants.
- NLP used for Review sentiment analysis

# Zomato API request

In [1]:
# Zomato web acesss
class Zomato:
    def __init__(self, user_key, content_type = 'application/json'):
        self._key = user_key
        self.request_count = 0
        self.content_type = content_type
        self._header = {
                    'Accept': content_type,
                    'user-key': user_key,
                            }
        self._categories_id = {}

    def get_response(self,api,params=''):
        import requests
        headers = self._header
        status = {  200 :'Sucessfully Connected to Zomato..',
                    403 :'Connection Error: Invalid User Keys',
                    404 : 'No Connection to Server..'
                }
        response = requests.get(api, headers=headers,params=params)
        print(status[response.status_code])
        if response.status_code == 200:
            self.request_count += 1
            return response.json()
    
    def categories(self,method='get'):
        api = 'https://developers.zomato.com/api/v2.1/categories'
        response = self.get_response(api)
        result_dict = {categories['categories']['name']: categories['categories']['id'] for categories in response['categories']}
        if method == 'set':
            self._categories_id = result_dict 
        return result_dict
    
    def cities(self,city_name='',id='',lat='',lon=''):
        params = (
            ('q', city_name),
            ('lat', lat),
            ('lon', lon),
            ('city_ids', id))

        api = 'https://developers.zomato.com/api/v2.1/cities'
        response = self.get_response(api,params=params)  
        return response

    def cuisines(self,city_id='',lat='',lon=''):
        params = (
            ('city_id', city_id),
            ('lat', lat),
            ('lon', lon))
        api = 'https://developers.zomato.com/api/v2.1/cuisines'
        response = self.get_response(api,params=params)  
        return response
    
    def geocode(self,lat='',lon=''):
        api = 'https://developers.zomato.com/api/v2.1/geocode'
        params = (
            ('lat', lat),
            ('lon', lon))
        response = self.get_response(api,params=params)  
        return response

    def location(self,city='',lat='',lon='',max_count=100):
        api = 'https://developers.zomato.com/api/v2.1/locations'
        params = (
        ('query', city),
        ('lat', lat),
        ('lon', lon),
        ('count', str(max_count)))
        response = self.get_response(api,params=params)  
        return response
    
    def location_details(self,city='',lat='',lon=''):
        r = self.location(city,lat,lon,1)
        entity_id = r['location_suggestions'][0]['entity_id']
        entity_type = r['location_suggestions'][0]['entity_type']

        params = (
            ('entity_id', entity_id),
            ('entity_type', entity_type))
        api = 'https://developers.zomato.com/api/v2.1/location_details'

        response = self.get_response(api,params=params)  
        return response
    
    def restaurant(self,res_id=''):
        api = 'https://developers.zomato.com/api/v2.1/restaurant'
        params = (
                ('res_id', res_id),)
        response = self.get_response(api,params=params) 
        return response
    
    def reviews(self,res_id='',start='1',count='10'):
        api = 'https://developers.zomato.com/api/v2.1/reviews'
        params = (
            ('res_id', res_id),
            ('start', start),
            ('count', count),) 
        response = self.get_response(api,params=params) 
        return response

In [2]:
# Importing Python Library
import pandas as pd #library for Dataframe
import numpy as np # Array 
import json #library for json file
import folium # Map Rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [5]:
# Kolkata's Coordinates

address = 'Kolkata, IN'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of **{}** are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of **Kolkata, IN** are 22.54541245, 88.3567751581234.


In [14]:
kolkata = pd.read_csv('kolkata_resturent.csv',index_col=0)
kolkata.head()

,pincode,latitude,longitude,location,subzone,top_cuisines,popularity,nearby_res
114,700122,22.766851,88.376411,Barrackpore,Barrackpore,"Chinese,North Indian,Fast Food,Bengali,Mughlai",4.17,"17983797,18311142,18698012,23973,18663023,2364..."
113,700121,22.766614,88.422983,Barrackpore,Barrackpore,"Chinese,North Indian,Fast Food,Bengali,Mughlai",4.17,"17983797,18311142,18698012,23973,18663023,2364..."
112,700120,22.752200,88.379000,Barrackpore,Barrackpore,"Chinese,North Indian,Fast Food,Bengali,Mughlai",4.17,"17983797,18311142,18698012,23973,18663023,2364..."
111,700119,22.734883,88.396685,Khardah,Khardah,"Chinese,North Indian,Fast Food,Biryani,Bengali",3.95,"19127565,24311,25341,19114341,18198698,1860263..."
109,700117,22.723511,88.375425,Khardah,Khardah,"Chinese,North Indian,Fast Food,Biryani,Bengali",3.95,"19127565,24311,25341,19114341,18198698,1860263..."


In [16]:
kolkata.dtypes

pincode           int64
latitude        float64
longitude       float64
location         object
subzone          object
top_cuisines     object
popularity      float64
nearby_res       object
dtype: object

In [15]:
map_kolkata = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, pincode in zip(kolkata['latitude'], kolkata['longitude'],kolkata['pincode']):
    label = '{}'.format(pincode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_kolkata)  
    
#map_kolkata

In [21]:

def generateBaseMap(default_location=[latitude, longitude], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

base_map = generateBaseMap()

In [27]:
from folium.plugins import HeatMap

base_map = generateBaseMap()
HeatMap(data=kolkata[['latitude', 'longitude', 'popularity']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist(), radius=12, max_zoom=13).add_to(base_map)
base_map